In [1]:
# Signs in Q1 and No purchase in Q1

import pandas as pd
import datetime
import numpy as np
import os


In [2]:
def recrusive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [3]:
New_Sign_Ups_file_list=list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
New_Sign_Ups_file_list=[x for x in New_Sign_Ups_file_list if "Master" in x]
df_new_sign_ups_Q1=pd.DataFrame({"file_path":New_Sign_Ups_file_list})
df_new_sign_ups_Q1['week_end_dt']=df_new_sign_ups_Q1['file_path'].apply(lambda x: x.split("y_weeks/MediaStorm_")[1][:10])
df_new_sign_ups_Q1=df_new_sign_ups_Q1[df_new_sign_ups_Q1['week_end_dt']>="2019-02-09"]
df_new_sign_ups_Q1=df_new_sign_ups_Q1[df_new_sign_ups_Q1['week_end_dt']<="2019-05-04"]
df_new_sign_ups_Q1.shape

(13, 2)

In [4]:
df_all_Q1_sign_ups=pd.DataFrame()
for file in df_new_sign_ups_Q1['file_path'].unique().tolist():
    df=pd.read_table(file,dtype=str,sep="|",usecols=["customer_id_hashed","email_address_hash","sign_up_date",'transaction_count','transaction_amount'])
    df_all_Q1_sign_ups=df_all_Q1_sign_ups.append(df)
print(df_all_Q1_sign_ups.shape)  
df_new_sign_up_ids=df_all_Q1_sign_ups[["customer_id_hashed","email_address_hash","sign_up_date"]]
id_list_Q1_new_sign_ups=df_all_Q1_sign_ups['customer_id_hashed'].unique().tolist() 


(1070123, 5)


In [5]:
daily_transactions_file_list=list(recrusive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_transactions_file_list=[x for x in daily_transactions_file_list if "aily" in x]
df_daily_sales_Q1=pd.DataFrame({"file_path":daily_transactions_file_list})

df_daily_sales_Q1['week_end_dt']=df_daily_sales_Q1['file_path'].apply(lambda x: x.split("y_weeks/MediaStorm_")[1][:10])
df_daily_sales_Q1=df_daily_sales_Q1[df_daily_sales_Q1['week_end_dt']>="2019-02-09"]
df_daily_sales_Q1=df_daily_sales_Q1[df_daily_sales_Q1['week_end_dt']<="2019-05-04"]
df_daily_sales_Q1.shape


(13, 2)

In [6]:
for file in df_daily_sales_Q1['file_path'].unique().tolist():
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=["location_id","customer_id_hashed","item_transaction_amt"])
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=["location_id","customer_id_hashed","subclass_transaction_amt"]).rename(columns={"subclass_transaction_amt":"item_transaction_amt"})
    df["item_transaction_amt"]=df["item_transaction_amt"].astype(float)
    df=df[df['item_transaction_amt']>0]
    del df['item_transaction_amt']
    df=df[~pd.isnull(df['customer_id_hashed'])].drop_duplicates("customer_id_hashed")
    df_new_sign_up_ids=pd.merge(df_new_sign_up_ids,df,on="customer_id_hashed",how="left")
    df_new_sign_up_ids=df_new_sign_up_ids[pd.isnull(df_new_sign_up_ids['location_id'])]
    del df_new_sign_up_ids['location_id']
    print(df_new_sign_up_ids.shape)

(996857, 3)
(919268, 3)
(839611, 3)
(749774, 3)
(664555, 3)
(583907, 3)
(501420, 3)
(425066, 3)
(303639, 3)
(229568, 3)
(165250, 3)
(98505, 3)
(15122, 3)


In [7]:
print(df_new_sign_up_ids.shape)
print(len(df_new_sign_up_ids['customer_id_hashed'].unique()))
print(len(df_new_sign_up_ids['email_address_hash'].unique()))


(15122, 3)
15122
15122


In [8]:
output=df_new_sign_up_ids[['customer_id_hashed','email_address_hash']]
output['segment']="New_Singed_and_No_Purchase_in_Q1"

In [9]:
Q1_weeks=sorted(df_daily_sales_Q1['week_end_dt'].unique().tolist())
Q1_weeks_2=sorted(df_new_sign_ups_Q1['week_end_dt'].unique().tolist())
Q1_weeks==Q1_weeks_2

True

In [17]:
df_new_no_purchase_same_week=pd.DataFrame()
for week_end_date in Q1_weeks:
    df_sales_file=df_daily_sales_Q1[df_daily_sales_Q1['week_end_dt']==week_end_date]['file_path'].tolist()[0]
    df_master_file=df_new_sign_ups_Q1[df_new_sign_ups_Q1['week_end_dt']==week_end_date]['file_path'].tolist()[0]
    
    df_master=pd.read_csv(df_master_file,usecols=["customer_id_hashed","email_address_hash","sign_up_date",'transaction_count','transaction_amount'],
                          dtype=str,sep="|")
    df_sales=pd.read_csv(df_sales_file,usecols=["customer_id_hashed","location_id"],dtype=str,sep="|")
    df_sales=df_sales.drop_duplicates()
    
    df_master_no_purchase=pd.merge(df_master,df_sales,on="customer_id_hashed",how="left")
    df_master_no_purchase=df_master_no_purchase[pd.isnull(df_master_no_purchase['location_id'])]
    
    df_new_no_purchase_same_week=df_new_no_purchase_same_week.append(df_master_no_purchase)

In [30]:
df_new_no_purchase_same_week.head(2)

,customer_id_hashed,email_address_hash,sign_up_date,transaction_count,transaction_amount,location_id
329,20e36fc48c4a17f829339c958f35310a6df02782b3a6b6...,e9c45b624567a2a6d82a3b3e7365dc85cc672b2a489aae...,2019-02-06,000000000000001.,0000000000000000.00,NaN
365,7c8a8cbdb610e9fbbb09f417d3c5f55beb60799f95aa74...,9917c9aa2076a71b772eb930a6ccdddacf3f9202d5cba5...,2019-02-05,000000000000001.,0000000000000000.00,NaN


In [19]:
print(len(df_new_no_purchase_same_week['customer_id_hashed'].unique()))

19472


In [20]:
df_master_no_purchase.shape

(1650, 6)

In [21]:
test=pd.merge(df_master,df_sales,on="customer_id_hashed",how="left")

In [22]:
test.shape

(86181, 6)

In [31]:
df_sales.shape

(1240796, 2)

In [25]:
test_2=pd.merge(df_sales,df_master,on="customer_id_hashed",how="left")
test_2.shape

(1240796, 6)

In [26]:
test_2.head(2)

,location_id,customer_id_hashed,email_address_hash,sign_up_date,transaction_count,transaction_amount
0,1671,c044127f7b4f5ce0e494f0200b97be309851269783476b...,NaN,NaN,NaN,NaN
1,5281,NaN,NaN,NaN,NaN,NaN


In [27]:
test_2[~pd.isnull(test_2['sign_up_date'])].shape

(84531, 6)

In [28]:
df_master.shape

(84615, 5)

In [9]:
Experian_control_files=list(recrusive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Experian_Email_Openers_Group/output_2nd_upload/"))
Experian_control_files=[x for x in Experian_control_files if "_C_" in x]
len(Experian_control_files)

10

In [11]:
Q2_regular_control_files=list(recrusive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_final_20190422/Test_and_Control/uploaded/"))
Q2_regular_control_files=[x for x in Q2_regular_control_files if os.path.basename(x)[:2]=="C_"]
len(Q2_regular_control_files)


25

In [13]:
all_control_files=Experian_control_files+Q2_regular_control_files

In [22]:
all_control_df=pd.DataFrame()

for file in all_control_files:
    try:
        df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','email_address_hash','segment'])
    except:
        df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','Email_Address_hash','segment']).rename(columns={"Email_Address_hash":"email_address_hash"})
    all_control_df=all_control_df.append(df)
    

In [35]:
aaa=pd.merge(output,all_control_df,on=['customer_id_hashed','email_address_hash'],how="left")

In [36]:
len(aaa['customer_id_hashed'].unique())

13761

In [32]:
summary_sign_date=df_new_sign_up_ids.groupby(['sign_up_date'])['customer_id_hashed'].count().to_frame().reset_index()
summary_sign_date=summary_sign_date.sort_values("sign_up_date")

In [36]:
output.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/NesSignUps_Q1_NoPurchase/BL_2019Q1_new_signups_no_purchase_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [37]:
summary_sign_date.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/NesSignUps_Q1_NoPurchase/summary_BL_2019Q1_new_signups_no_purchase_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

